# **<mark>Transaction</mark>**

Giao dịch (Transaction) là đơn vị công việc logic thực hiện một hoạt động đơn lẻ hoặc nhiều hoạt động trong cơ sở dữ liệu. Các giao dịch có thể bao gồm một thao tác đọc, ghi, xóa hoặc cập nhật hoặc kết hợp các thao tác này.

Giả sử khi chúng ta muốn rút tiền từ máy ATM, ứng dụng ATM sẽ thực hiện thao tác này qua ba bước. Bước đầu tiên, ứng dụng sẽ kiểm tra số dư của tài khoản, sau đó nó sẽ trừ tiền từ tài khoản nguồn. Cùng với hai quy trình này, nó sẽ lưu giữ nhật ký của hoạt động rút tiền này. Hình ảnh sau đây minh họa cơ bản nguyên lý hoạt động của các giao dịch trong hệ thống cơ sở dữ liệu quan hệ.

![Demo](illustration-of-the-transactions-in-sql-server_grey.png)

Ý tưởng chính của các giao dịch là khi mỗi câu lệnh trả về lỗi, toàn bộ sửa đổi sẽ khôi phục lại để cung cấp tính toàn vẹn của dữ liệu. Mặt khác, nếu tất cả các câu lệnh được hoàn thành thành công, việc sửa đổi dữ liệu sẽ trở thành vĩnh viễn trên cơ sở dữ liệu. Do đó, nếu gặp bất kỳ sự cố mất điện hoặc các sự cố khác trong quá trình rút tiền từ máy ATM, các giao dịch đảm bảo tính nhất quán số dư. Đây sẽ là phương pháp tốt nhất để thực hiện tất cả các bước này thông qua một giao dịch bởi vì bốn thuộc tính chính của giao dịch cho phép tất cả các hoạt động chính xác và nhất quán hơn.

Tất cả các thuộc tính này được gọi là **ACID (Atomicity, Consistency, Isolation, Durable)** trong hệ thống cơ sở dữ liệu quan hệ với chữ cái đầu tiên của tên của chúng.

- **Atomicity:** Toàn bộ các hoạt động được bao gồm trong giao dịch được thực hiện thành công. Nếu không, tất cả các hoạt động sẽ bị hủy tại điểm xảy ra lỗi và tất cả các hoạt động trước đó được khôi phục lại
- **Consistency:** Thuộc tính này đảm bảo rằng tất cả dữ liệu sẽ nhất quán sau khi hoàn thành giao dịch theo các quy tắc, ràng buộc, tầng và trình kích hoạt đã xác định
- **Isolation:** Tất cả các giao dịch được cách ly với các giao dịch khác
- **Durable:** Việc sửa đổi các giao dịch đã cam kết vẫn tồn tại trong cơ sở dữ liệu

In [1]:
CREATE TABLE Person (
    PersonID int PRIMARY KEY IDENTITY(1,1),
    LastName varchar(255),
    FirstName varchar(255),
    Address varchar(255),
    City varchar(255),
	Age INT
)
 
GO
 
INSERT INTO Person VALUES('Hayes', 'Corey','123  Wern Ddu Lane','LUSTLEIGH',23)
INSERT INTO Person VALUES('Macdonald','Charlie','23  Peachfield Road','CEFN EINION',45)
INSERT INTO Person VALUES('Frost','Emma','85  Kingsway North','HOLTON',26)
INSERT INTO Person VALUES('Thomas', 'Tom','59  Dover Road', 'WESTER GRUINARDS',51)
INSERT INTO Person VALUES('Baxter','Cameron','106  Newmarket Road','HAWTHORPE',46)
INSERT INTO Person VALUES('Townsend','Imogen ','100  Shannon Way','CHIPPENHAM',20)
INSERT INTO Person VALUES('Preston','Taylor','14  Pendwyallt Road','BURTON',19)
INSERT INTO Person VALUES('Townsend','Imogen ','100  Shannon Way','CHIPPENHAM',18)
INSERT INTO Person VALUES('Khan','Jacob','72  Ballifeary Road','BANCFFOSFELEN',11)

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.026

## **<mark>Chế độ giao dịch trong SQL Server</mark>**

SQL Server có thể vận hành 3 chế độ giao dịch khác nhau và đó là:

- Chế độ giao dịch tự động gửi (Autocommit Transaction mode) là chế độ giao dịch mặc định cho SQL Server. Trong chế độ này, mỗi câu lệnh T-SQL được đánh giá như một giao dịch và chúng được cam kết thực thi hoặc quay trở lại tùy theo kết quả của chúng. Các câu lệnh thành công được cam kết và các câu lệnh không thành công được khôi phục ngay lập tức
- Chế độ giao dịch ngầm định (Implicit transaction mode) cho phép SQL Server bắt đầu một giao dịch ngầm định cho mọi câu lệnh DML nhưng chúng ta cần sử dụng các lệnh cam kết hoặc lệnh quay lại một cách rõ ràng ở cuối các câu lệnh
- Chế độ giao dịch rõ ràng (Explicit transaction mode) cung cấp để xác định một giao dịch chính xác với điểm bắt đầu và điểm kết thúc của giao dịch

### **<mark>Cách xác định một giao dịch ngầm định trong SQL Server</mark>**

In [ ]:
SET IMPLICIT_TRANSACTIONS ON
SELECT * FROM Person
WHERE PersonID = 2

UPDATE 
    Person 
SET 
    Lastname = 'Sawyer', 
    Firstname = 'Tom' 
WHERE 
    PersonID = 2

SELECT 
    IIF(@@OPTIONS = 5498, 
    'Implicit Transaction Mode ON', 
    'Implicit Transaction Mode OFF'
    ) AS 'Transaction Mode'

SELECT * FROM Person
WHERE PersonID = 2


SELECT 
    @@TRANCOUNT AS OpenTransactions

ROLLBACK TRAN

SELECT * FROM Person
WHERE PersonID = 2

SELECT 
    @@TRANCOUNT AS OpenTransactions

COMMIT TRAN

SELECT 
    @@TRANCOUNT AS OpenTransactions

\- @@OPTIONS: [@@OPTIONS (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/functions/options-transact-sql?view=sql-server-ver16)

\- Bitwise &: [& (Bitwise AND) (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/language-elements/bitwise-and-transact-sql?view=sql-server-ver16)

\- @@TRANCOUNT: [@@TRANCOUNT (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/vi-vn/sql/t-sql/functions/trancount-transact-sql?view=sql-server-2017)

### **<mark>Cách xác định một giao dịch rõ ràng trong SQL Server</mark>**

Để xác định một giao dịch rõ ràng, chúng tôi bắt đầu sử dụng lệnh BEGIN TRANSACTION vì câu lệnh này xác định điểm bắt đầu của giao dịch rõ ràng. Nó có cú pháp sau:

```
BEGIN TRANSACTION [ {transaction_name | @tran_name_variable }    
    [WITH MARK ['description']]]

```

- Tùy chọn transaction\_name được sử dụng để chỉ định một tên cụ thể cho các giao dịch
- Tùy chọn @trans\_var là một biến do người dùng xác định được sử dụng để giữ tên giao dịch
- Tùy chọn WITH MARK cho phép đánh dấu một giao dịch cụ thể trong tệp nhật ký

Sau khi xác định một giao dịch rõ ràng thông qua lệnh BEGIN TRANSACTION, các tài nguyên liên quan có được một khóa tùy thuộc vào mức độ cô lập của giao dịch. Vì lý do này, việc sử dụng giao dịch ngắn nhất có thể sẽ giúp giảm thiểu các vấn đề về khóa. Câu lệnh sau đây bắt đầu một giao dịch và sau đó nó sẽ thay đổi tên của một hàng cụ thể trong bảng Person.

Câu lệnh COMMIT TRAN áp dụng các thay đổi dữ liệu cho cơ sở dữ liệu và dữ liệu được thay đổi sẽ trở thành vĩnh viễn.

In [ ]:
BEGIN TRAN
UPDATE Person 
SET    Lastname = 'Lucky', 
        Firstname = 'Luke' 
WHERE  PersonID = 1
SELECT @@TRANCOUNT AS OpenTransactions 
COMMIT TRAN 
SELECT @@TRANCOUNT AS OpenTransactions

Câu lệnh <mark>**ROLLBACK TRANSACTION**</mark> giúp hoàn tác tất cả các sửa đổi dữ liệu được áp dụng bởi giao dịch.

In [ ]:
BEGIN TRAN
UPDATE Person 
SET    Lastname = 'Donald', 
        Firstname = 'Duck'  WHERE PersonID=2
 
 
SELECT * FROM Person WHERE PersonID=2
 
ROLLBACK TRAN 
 
SELECT * FROM Person WHERE PersonID=2

Bảng sau minh họa cấu trúc của các giao dịch rõ ràng trong SQL Server.

![Demo2](Table1.png)

### **<mark>Lưu các điểm (vị trí) trong các giao dịch</mark>**

Các điểm lưu có thể được sử dụng để khôi phục bất kỳ phần cụ thể nào của giao dịch thay vì toàn bộ giao dịch. Vì vậy, chỉ có thể khôi phục bất kỳ phần nào của giao dịch trong khoảng thời gian từ sau điểm lưu đến trước lệnh khôi phục. Để xác định điểm lưu trong một giao dịch, sử dụng cú pháp SAVE TRANSACTION và sau đó  thêm tên cho điểm lưu.

In [ ]:
BEGIN TRANSACTION 
INSERT INTO Person 
VALUES('Mouse', 'Micky','500 South Buena Vista Street, Burbank','California',43)
SAVE TRANSACTION InsertStatement
DELETE Person WHERE PersonID=3
SELECT * FROM Person 
ROLLBACK TRANSACTION InsertStatement
COMMIT
SELECT * FROM Person

## **<mark>Giao dịch tự động khôi phục trong SQL Server</mark>**

Thông thường, các giao dịch bao gồm nhiều hơn một truy vấn. Theo cách này, nếu một trong các câu lệnh SQL trả về lỗi, tất cả các sửa đổi sẽ bị xóa và các câu lệnh còn lại không được thực thi. Quá trình này được gọi là Giao dịch khôi phục tự động trong SQL.

In [ ]:
BEGIN TRAN
INSERT INTO Person 
VALUES('Bunny', 'Bugs','742 Evergreen Terrace','Springfield',54)
    
UPDATE Person SET Age='MiddleAge' WHERE PersonID=7
SELECT * FROM Person
 
COMMIT TRAN